In [222]:
import pandas as pd
from PIL import Image
import cv2
import numpy as np
import math

In [251]:
original_images = context.catalog.load("express_expense")

[05/28/23 20:46:21] INFO     Loading data from 'express_expense' (PartitionedDataSet)...        ]8;id=334628;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=726864;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [252]:
points_auto = context.catalog.load("homography_points_segmentation_batched")

                    INFO     Loading data from 'homography_points_segmentation_batched'         ]8;id=994361;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=402258;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (CSVDataSet)...                                                                       

In [253]:
points_auto

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
0,batched_1000,181,198,662,178,709,999,111,999
1,batched_1001,89,49,551,80,617,902,71,948
2,batched_1002,83,25,415,181,370,999,2,999
3,batched_1003,183,68,597,47,585,990,156,942
4,batched_1004,83,71,601,76,662,906,17,889
...,...,...,...,...,...,...,...,...,...
185,batched_1195,122,0,579,0,623,999,65,999
186,batched_1196,37,0,288,0,290,347,16,347
187,batched_1197,25,117,286,101,309,449,21,436
188,batched_1198,6,0,220,13,232,449,1,449


In [254]:
points_auto = points_auto.rename(columns={'index': 'Image'})
# points_auto["Image"] = points_auto['Image'].str.replace("-receipt.jpg", "").astype('uint32')
points_auto["Image"] = points_auto['Image'].str.replace("batched_", "").astype('uint32')
points_auto = points_auto.set_index("Image")

In [255]:
points_auto

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
Image,,,,,,,,
1000,181,198,662,178,709,999,111,999
1001,89,49,551,80,617,902,71,948
1002,83,25,415,181,370,999,2,999
1003,183,68,597,47,585,990,156,942
1004,83,71,601,76,662,906,17,889
...,...,...,...,...,...,...,...,...
1195,122,0,579,0,623,999,65,999
1196,37,0,288,0,290,347,16,347
1197,25,117,286,101,309,449,21,436


In [256]:
points_manual = pd.read_csv("../data/01_raw/homography.csv", sep=";")

In [257]:
points_manual['P1X'] = points_manual['P1'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [258]:
points_manual['P1Y'] = points_manual['P1'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [259]:
points_manual['P2X'] = points_manual['P2'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [260]:
points_manual['P2Y'] = points_manual['P2'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [261]:
# o P3 está invertido com o P4 na detecção manual, não sei por qual motivo

In [262]:
points_manual['P3X'] = points_manual['P4'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [263]:
points_manual['P3Y'] = points_manual['P4'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [264]:
points_manual['P4X'] = points_manual['P3'].str.split(',').apply(lambda x: x[0]).astype('uint32')

In [265]:
points_manual['P4Y'] = points_manual['P3'].str.split(',').apply(lambda x: x[1]).astype('uint32')

In [266]:
points_manual = points_manual.drop(columns=['P1', 'P2', 'P3', 'P4'])

In [267]:
points_manual = points_manual.set_index("Image")

### Apenas 73% das imagens foram reconhecidas automaticamente

In [268]:
missing = set(points_manual.index) - set(points_auto.index)
missing

{1006, 1038, 1097, 1146, 1147, 1163, 1167, 1176, 1187}

In [269]:
points_manual = points_manual.loc[points_auto.index]

In [270]:
points_manual

,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y
Image,,,,,,,,
1000,182,199,661,193,699,929,119,914
1001,83,77,545,90,576,788,72,824
1002,76,110,413,179,355,998,1,981
1003,177,92,596,92,594,932,159,943
1004,83,94,603,101,659,851,21,875
...,...,...,...,...,...,...,...,...
1195,117,7,583,7,623,996,65,994
1196,36,1,289,2,290,346,17,346
1197,25,116,287,101,308,445,19,437


**Problemas em calcular a diferença entre os pontos manuais e os automáticos**
1. A distância entre dois pontos deve ser considerada a mais curta, do teorema de Pitágoras?
2. As imagens têm resoluções diferentes, o que impacta nessa conta das métricas

Interessante mencionar no artigo. Dito isso, vamos aos cálculos:

1. Diferença entre os pontos manuais e os pontos automáticos (Pitágoras)

In [271]:
dp1 = np.sqrt(pow(points_auto['P1X'] - points_manual['P1X'], 2) + pow(points_auto['P1Y'] - points_manual['P1Y'], 2))
dp2 = np.sqrt(pow(points_auto['P2X'] - points_manual['P2X'], 2) + pow(points_auto['P2Y'] - points_manual['P2Y'], 2))
dp3 = np.sqrt(pow(points_auto['P3X'] - points_manual['P3X'], 2) + pow(points_auto['P3Y'] - points_manual['P3Y'], 2))
dp4 = np.sqrt(pow(points_auto['P4X'] - points_manual['P4X'], 2) + pow(points_auto['P4Y'] - points_manual['P4Y'], 2))

In [272]:
dt = np.array(np.ravel(np.concatenate((dp1, dp2, dp3, dp4)).reshape(-1)))

In [273]:
dt.max()

451.0277153346566

In [274]:
dt.min()

0.0

In [275]:
dt.mean()

21.727019906824825

In [276]:
np.median(dt)

6.363839778884804

In [277]:
dt.std()

51.39553560040647

In [250]:
np.argmax(dt)

685